In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
import pickle

In [2]:
data=pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [4]:
lable_encoder_gender=LabelEncoder()
data['Gender']=lable_encoder_gender.fit_transform(data['Gender'])

In [5]:
onehot_encoder_geo=OneHotEncoder()
geo_encoder=onehot_encoder_geo.fit_transform(data[['Geography']])
onehot_encoder_geo.get_feature_names_out(['Geography'])
geo_encoded_df=pd.DataFrame(geo_encoder.toarray(),columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [6]:
data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [7]:
x=data.drop('EstimatedSalary',axis=1)
y=data['EstimatedSalary']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [8]:
scalar=StandardScaler()
x_train_scaled=scalar.fit_transform(x_train)
x_test_scaled=scalar.transform(x_test)

In [9]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scalar,file)
    
with open('lable_encoder_gender.pkl','wb') as file:
    pickle.dump(lable_encoder_gender,file)
    
with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)

In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [11]:
## build the
model=Sequential([
    Dense(64,activation='relu',input_shape=(x_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1)
])

## compile the model
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])
model.summary()




Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

log_dir="regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [13]:
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [14]:
## train the model
history=model.fit(
    x_train_scaled,y_train,
    validation_data=(x_test,y_test),
    epochs=100,
    callbacks=[early_stopping_callback,tensorboard_callback]
    )

Epoch 1/100


250/250 [==============================] - 15s 33ms/step - loss: 100378.6328 - mae: 100378.6328 - val_loss: 3532063.0000 - val_mae: 3532063.0000
Epoch 2/100
250/250 [==============================] - 2s 7ms/step - loss: 99675.8984 - mae: 99675.8984 - val_loss: 29432206.0000 - val_mae: 29432206.0000
Epoch 3/100
250/250 [==============================] - 2s 9ms/step - loss: 97224.2109 - mae: 97224.2109 - val_loss: 99398544.0000 - val_mae: 99398544.0000
Epoch 4/100
250/250 [==============================] - 2s 7ms/step - loss: 92396.9688 - mae: 92396.9688 - val_loss: 224925680.0000 - val_mae: 224925680.0000
Epoch 5/100
250/250 [==============================] - 2s 8ms/step - loss: 85296.7344 - mae: 85296.7344 - val_loss: 409919072.0000 - val_mae: 409919072.0000
Epoch 6/100
250/250 [==============================] - 2s 7ms/step - loss: 76764.5703 - mae: 76764.5703 - val_loss: 642141184.0000 - val_mae: 642141184.0000
Epoch 7/100
250/250 [==============================] - 2s 7m

In [15]:
%load_ext tensorboard

In [16]:
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6006 (pid 4192), started 1:58:44 ago. (Use '!kill 4192' to kill it.)

In [17]:
## evaluate the model on test data
test_loss,test_mae=model.evaluate(x_test_scaled,y_test)
print(f"Test MAE: {test_mae}")
    

63/63 [==============================] - 1s 3ms/step - loss: 98524.2656 - mae: 98524.2656
Test MAE: 98524.265625


In [18]:
## save the model
model.save('salary_regression_model.h5')

d:\velocity\GENARATIVE AI\ANN Classification\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
